In [1]:
import pandas as pd
import numpy as np
import time
import pickle
import operator
from collections import defaultdict
import timeit
# %run searchModel.ipynb

This evaluation uses multiprocessing in order to speed it up. This libary in combination with ipython only works on Linux.

In [3]:
class Evaluation:
    
    qrel_file={}

    def __init__(self, qrel_file_path):
        self.qrel_file=pickle.load(open(qrel_file_path, 'rb'))
#         self.qrel_file = pd.read_csv(qrel_file_path)

    def precisionEvaluation(self, quer_id, revelance_doc_id:list):
        count=len(revelance_doc_id)
        countRelevanceSearch=0
        for doc_id in revelance_doc_id:
            if doc_id in self.qrel_file[quer_id]:
                countRelevanceSearch=countRelevanceSearch+1
        if count!=0:
            precision=countRelevanceSearch/count
        else:
            precision=0
        return precision
    
    def recallEvluation(self,query_id, revelance_doc_id):
            count=0
            countRelevance=len(self.qrel_file[query_id])
            for doc_id in revelance_doc_id:
                if doc_id in self.qrel_file[query_id]:
                    count=count+1
            recall= count/countRelevance
            return recall
    
    def Fscore(self,query_id,revelance_doc_id,beta):
        precision=self.precisionEvaluation(query_id,revelance_doc_id)
        recall=self.precisionEvaluation(query_id,revelance_doc_id)
        Fscore=((np.square(beta)+1)*precision*recall)/(np.square(beta)*precision+recall)
        return Fscore
    
    def AveragePercision(self, query_id, revelance_doc_id):
        totalprecision=0.
        count_predictRight=0
        for i, doc_id in enumerate(revelance_doc_id):
            if doc_id in self.qrel_file[query_id]:
                count_predictRight=count_predictRight+1
                position = 0
                position = i+1
#                 for i in range(len(self.qrel_file[query_id])):
#                     if self.qrel_file[query_id][i] == doc_id:
#                         position = i+1                
                    
#                 position=np.where(revelance_doc_id==doc_id)[0]+1
                totalprecision=totalprecision+count_predictRight/position

        print('PPPPPPPPPPPPPPPPPP:',len(revelance_doc_id)/count_predictRight if count_predictRight>0 else 0)
        if count_predictRight==0:
            AveragePercision=0
        else:
            AveragePercision=totalprecision/count_predictRight
        return AveragePercision
    
    def Precsion_at_K(self, query_id,revelance_doc_id,K):
        top_K_document=list()
        for doc_id in range(K[0]):
            top_K_document.append(revelance_doc_id[doc_id])
            Precsion_at_K=self.precisionEvaluation(query_id,top_K_document)
        return Precsion_at_K
    
    ##NDCG是一种衡量排序质量的评价指标，该指标考虑了所有元素的相关性
    # https://blog.csdn.net/waple_0820/article/details/52014209 as python example
    #https: // www.cnblogs.com / HappyAngel / p / 3535919.html
    #NDCG=DCG/IDCG
#https://en.wikipedia.org/wiki/Discounted_cumulative_gain
    def dcg(self,query_id,revelance_doc_id):
        def get_relevence(query_id, doc_id):
            gs = self.qrel_file[query_id]
            if doc_id in gs:
                return 1
            else:
                return 0
        i=1
        dcg_accumulating =0
        for doc_id in revelance_doc_id:
            discount_factor=1/np.log(max(i,2))
            revelance=get_relevence(query_id,doc_id)
            gain=np.multiply(discount_factor,revelance)
            dcg_accumulating=dcg_accumulating+gain
            i=i+1
        return dcg_accumulating
    
    def idcg(self,query_id):
        haveDoc_id=self.qrel_file[query_id]
        sortDoc_id=sorted(haveDoc_id,key=operator.itemgetter(1),reverse=True)
        idcg=self.dcg(query_id,sortDoc_id)
        return idcg

    def get_relevence(self, query_id, doc_id):
        try:
            score = self.qrel_file[query_id][doc_id]
        except KeyError as error:
            score==0
        return score

    def ndcg(self,query_id,revelance_doc_id):
        dcg=self.dcg(query_id,revelance_doc_id)
        idcg = self.idcg(query_id)
        if idcg==0:
            idcg=idcg+1
        return dcg/idcg
    
    def Evaluation(self,query,model, model_type,number_documents):
        time_sum=0
        count=0
        averagePrecison=0.
        Ndcg=0
        result = []

        sumPrecison = 0.
        for index,row in query.iterrows():
            query_id=row['id']
            query_text=row['text']
            rowResult = {}
            start_time=time.time()
            ##### main method called search to intergate all the methods
            revelance_doc_id=model.query(model_type, query_text, number_documents)
            calculation_time=time.time()-start_time
            revelance_doc_id = list(revelance_doc_id)
            time_sum=time_sum+calculation_time
            if len(revelance_doc_id) == 0:
                continue
            if query_id not in self.qrel_file:
                continue
            averagePrecison=self.AveragePercision(query_id,revelance_doc_id)
            sumPrecison += averagePrecison
            Ndcg=self.ndcg(query_id,revelance_doc_id)
            count=count+1
            ####check if its running
            if count % 500==0:
                print(" not crushed",count)
            #mean_averagePrecision=sumPrecison/count
            #mean_ndcg=Ndcg/count
            #avg_time = time_sum / count
            print('query_id:',query_id)
            #print("\tAverage Precision = {}".format(averagePrecison))
            #print("\tNdcg={}".format(mean_ndcg))
            #print('\taverage time ={}'.format(avg_time))
            result.append((query_id,averagePrecison,Ndcg,calculation_time))
            #result.loc[len(result)] = rowResult
        labels = ['id','avgPrec','NDCG','time']
        rtn = pd.DataFrame.from_records(result, columns=labels)
        
        return rtn

In [14]:
%run utils.ipynb
%run preprocessing_ti.ipynb
%run basicVSM.ipynb
%run sparseBasicVSM.ipynb
%run SparsePreclustering.ipynb
%run randomProjections.ipynb
%run SearchModel_ti.ipynb
class SearchModel:
    
    def __init__(self):
        print('Building SearchModel')
        print('Building TieredIndex...')
        inverted_index_path = 'data/inverted_index.p'
        global_tier_path = 'data/global_tier.p'
        number_tier_path = 'data/number_tier.p'
        
        self.stop_word_file_path = 'data/nfcorpus/raw/stopwords.large'
        self.rare_words_file_path = 'data/rare_tokens.txt'
        #params for Jonas
        collection_processed_path = 'data/devProcessed.psave'
        inverted_index_ori_path = 'data/invertedIndexDevOri.psave'
        sparse_vsm_path = 'data/sparseVSM.psave'
        random_projection_hash_path = 'data/randomProjectionN1500HashTrue.psave'
        random_projection_NoHash_path = 'data/randomProjectionN1500HashFalse.psave'

        preclustering_path = 'data/preclustering.psave'
        
        s = SearchModel_ti(inverted_index_path, global_tier_path,number_tier_path, self.stop_word_file_path,self.rare_words_file_path)
        gs_path = '../data/dev_gs.p'
        self.tiered_index = s.init__search_model('TieredIndex')
        
        #self.docs_preprocessed = preprocessing(load=True)
        self.docs_preprocessed  = pickle.load(open('preprocessing.psave','rb'))['docs_preprocessed']
        
        #load dataframes
        with open(collection_processed_path,"rb") as f:
            self.collectionProcessed = pickle.load(f)
        #set processing to load
        print('Creating InvertedIndex...')
        #load index
        self.invertedIndex = createInvertedIndex(self.collectionProcessed,load=True,path=inverted_index_ori_path)
        print('Creating IDF...')
        self.idf = createIDF(self.invertedIndex,len(self.collectionProcessed))
        print('Setup sparse VSM...')
        self.sVSM = SparseBasicVSM(self.collectionProcessed,self.invertedIndex, load=True, path=sparse_vsm_path)
        print('load preclustering ...')
        with open(preclustering_path,"rb") as f:
            self.precluster = pickle.load(f)
        #self.precluster = SparsePreclustering(self.collectionProcessed,self.invertedIndex,self.idf,load=True,path=preclustering_path)
        print('load random Projection...')
        with open(random_projection_hash_path,"rb") as f:
            self.randomProjectionHash = pickle.load(f)
        with open(random_projection_NoHash_path,"rb") as f:
            self.randomProjectionNoHash = pickle.load(f)
        
    def query(self,model:str, query:str, num_des:int):
        """
        
        """
        results = []
        if model == 'TieredIndex':
            results = self.tiered_index.search(query,num_des, self.invertedIndex, self.docs_preprocessed, self.idf)
        else:
            p = Preprocessor(self.stop_word_file_path,self.rare_words_file_path)
            query =  p.preprocess_line(query)
            query = list(set(query.split()))
            if model == 'BasicVSM':
                results = retrieveCosine(query,self.invertedIndex,self.idf,self.collectionProcessed,num_des)
            if model == 'CosineSparse':
                results = self.sVSM.retrieveCosineSparse(query,num_des)
            if model == 'Preclustering':
                results = self.precluster.retrieveWithPreclustering(query, num_des)
            if model == 'RandomProjectionNoHash': 
                results = self.randomProjectionNoHash.retrieveWithRandom(query, amount=num_des)
            if model == 'RandomProjectionHash': 
                results = self.randomProjectionHash.retrieveWithRandom(query, amount=num_des)
            #results = [self.docs_preprocessed[results[i][0]][0] for i in range(min(len(results),num_des))]
            results = [a[0] for a in results]
        return results

In [18]:
#evaluation of the tiered Index
qrel_file_path=r'data/dev_gs.p'
query_df = pd.read_csv('data/dev.all.queries.df')
e = Evaluation(qrel_file_path)
s = SearchModel()
resTi = e.Evaluation(query_df,model=s, model_type='TieredIndex',number_documents=80)
resTi.to_csv('resultsTI.df')

Building SearchModel
Building TieredIndex...
data/nfcorpus/raw/stopwords.large
Creating InvertedIndex...
Creating IDF...
Setup sparse VSM...
load preclustering ...
load random Projection...
PPPPPPPPPPPPPPPPPP: 66.6774193548387
query_id: PLAIN-1
PPPPPPPPPPPPPPPPPP: 27.045454545454547
query_id: PLAIN-1007
PPPPPPPPPPPPPPPPPP: 106.8695652173913
query_id: PLAIN-101
PPPPPPPPPPPPPPPPPP: 41.391304347826086
query_id: PLAIN-1017
PPPPPPPPPPPPPPPPPP: 15.833333333333334
query_id: PLAIN-1027
PPPPPPPPPPPPPPPPPP: 51.45454545454545
query_id: PLAIN-1038
PPPPPPPPPPPPPPPPPP: 468.0
query_id: PLAIN-1049
PPPPPPPPPPPPPPPPPP: 333.0
query_id: PLAIN-1065
PPPPPPPPPPPPPPPPPP: 194.66666666666666
query_id: PLAIN-1077
PPPPPPPPPPPPPPPPPP: 62.75
query_id: PLAIN-1087
PPPPPPPPPPPPPPPPPP: 321.0
query_id: PLAIN-1097
PPPPPPPPPPPPPPPPPP: 51.87234042553192
query_id: PLAIN-11
PPPPPPPPPPPPPPPPPP: 468.0
query_id: PLAIN-1108
PPPPPPPPPPPPPPPPPP: 51.89655172413793
query_id: PLAIN-111
PPPPPPPPPPPPPPPPPP: 52.4
query_id: PLAIN-1118
PP

PPPPPPPPPPPPPPPPPP: 402.5
query_id: PLAIN-2489
PPPPPPPPPPPPPPPPPP: 262.6
query_id: PLAIN-2499
PPPPPPPPPPPPPPPPPP: 74.55882352941177
query_id: PLAIN-2509
PPPPPPPPPPPPPPPPPP: 120.8
query_id: PLAIN-2519
PPPPPPPPPPPPPPPPPP: 115.63636363636364
query_id: PLAIN-2529
PPPPPPPPPPPPPPPPPP: 69.72222222222223
query_id: PLAIN-2539
PPPPPPPPPPPPPPPPPP: 324.375
query_id: PLAIN-2549
PPPPPPPPPPPPPPPPPP: 93.56521739130434
query_id: PLAIN-2559
PPPPPPPPPPPPPPPPPP: 64.27777777777777
query_id: PLAIN-2569
PPPPPPPPPPPPPPPPPP: 77.63636363636364
query_id: PLAIN-2579
PPPPPPPPPPPPPPPPPP: 259.3333333333333
query_id: PLAIN-258
PPPPPPPPPPPPPPPPPP: 46.673469387755105
query_id: PLAIN-2589
PPPPPPPPPPPPPPPPPP: 40.1551724137931
query_id: PLAIN-2599
PPPPPPPPPPPPPPPPPP: 28.3875
query_id: PLAIN-2609
PPPPPPPPPPPPPPPPPP: 139.16666666666666
query_id: PLAIN-2619
PPPPPPPPPPPPPPPPPP: 29.25
query_id: PLAIN-2629
PPPPPPPPPPPPPPPPPP: 38.851851851851855
query_id: PLAIN-2639
PPPPPPPPPPPPPPPPPP: 98.56
query_id: PLAIN-2649
PPPPPPPPPPPPPPPP

PPPPPPPPPPPPPPPPPP: 322.6666666666667
query_id: PLAIN-859
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-871
PPPPPPPPPPPPPPPPPP: 5.396039603960396
query_id: PLAIN-881
PPPPPPPPPPPPPPPPPP: 124.4
query_id: PLAIN-891
PPPPPPPPPPPPPPPPPP: 46.166666666666664
query_id: PLAIN-90
PPPPPPPPPPPPPPPPPP: 91.5
query_id: PLAIN-901
PPPPPPPPPPPPPPPPPP: 256.0
query_id: PLAIN-912
PPPPPPPPPPPPPPPPPP: 17.308823529411764
query_id: PLAIN-923
PPPPPPPPPPPPPPPPPP: 28.666666666666668
query_id: PLAIN-933
PPPPPPPPPPPPPPPPPP: 8.29126213592233
query_id: PLAIN-945
PPPPPPPPPPPPPPPPPP: 52.411764705882355
query_id: PLAIN-955
PPPPPPPPPPPPPPPPPP: 141.33333333333334
query_id: PLAIN-965
PPPPPPPPPPPPPPPPPP: 212.66666666666666
query_id: PLAIN-976
PPPPPPPPPPPPPPPPPP: 67.83333333333333
query_id: PLAIN-986
PPPPPPPPPPPPPPPPPP: 353.0
query_id: PLAIN-996


In [5]:
import multiprocessing as mp

In [ ]:
def rowFunction(model_type):
    print(model_type)
    res = e.Evaluation(query_df,model=s, model_type=model_type,number_documents=80)
    res['model']=model_type
    return res
def rowFunction2(queries):
    #print(model_type)
    res = e.Evaluation(queries,model=s, model_type='BasicVSM',number_documents=80)
    res['model']='BasicVSM'
    return res

In [23]:
#evaluation of the other files, multiprocessed. Attention, only works on linux machines.
totalStart = time.time()
pool = mp.Pool(processes=3)
result_df=[]
models=['CosineSparse','Preclustering','RandomProjectionNoHash','RandomProjectionHash']
qrel_file_path=r'data/dev_gs.p'
query_df = pd.read_csv('data/dev.all.queries.df')
q_df1 = query_df.head(100)
q_df3 = query_df.tail(100)
q_df2 = query_df.truncate(before=100,after = len(query_df)-101)
q = [q_df1,q_df2,q_df3]
e = Evaluation(qrel_file_path)
s = SearchModel()
#run basic VSM, takes about 7h

result_df = pool.map(rowFunction2,q)
final=pd.concat(result_df)
final.to_csv('resultsBasic.df')
totalTime1 = time.time() - totalStart
totalStart = time.time()

#run other models, takes about 2h
result_df = pool.map(rowFunction,models)
final=pd.concat(result_df)
final.to_csv('resultsOthers.df')
totalTime2 = time.time() - totalStart


Building SearchModel
Building TieredIndex...
data/nfcorpus/raw/stopwords.large
Creating InvertedIndex...
Creating IDF...
Setup sparse VSM...
load preclustering ...
load random Projection...
RandomProjectionNoHash
Preclustering
CosineSparse
PPPPPPPPPPPPPPPPPP: 16.0
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-1
query_id: PLAIN-1
PPPPPPPPPPPPPPPPPP: 4.444444444444445
query_id: PLAIN-1007
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-1
PPPPPPPPPPPPPPPPPP: 11.428571428571429
query_id: PLAIN-1007
PPPPPPPPPPPPPPPPPP: 8.0
query_id: PLAIN-1007
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-101
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1017
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-1027
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1038
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1049
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1065
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1077
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-1087
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1097
PPPPPPPPPPPPPPPPPP: 26.6666

PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-1408
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1982
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1994
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2007
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2018
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2029
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1418
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2039
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2050
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-1202
PPPPPPPPPPPPPPPPPP: 6.666666666666667
query_id: PLAIN-142
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1213
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-206
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2060
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2070
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2080
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2091
PPPPPPPPPPPPPPPPPP: 8.88888888888889
query_id: PLAIN-1428
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2101
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2112
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2123
PPPPPPPPPPPP

PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-216
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1578
PPPPPPPPPPPPPPPPPP: 2.962962962962963
query_id: PLAIN-2166
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2459
PPPPPPPPPPPPPPPPPP: 5.0
query_id: PLAIN-2176
PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-1589
PPPPPPPPPPPPPPPPPP: 2.857142857142857
query_id: PLAIN-2186
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1600
PPPPPPPPPPPPPPPPPP: 7.2727272727272725
query_id: PLAIN-2196
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-1610
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1620
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-1634
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-164
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-1644
PPPPPPPPPPPPPPPPPP: 2.5
query_id: PLAIN-1655
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-22
PPPPPPPPPPPPPPPPPP: 11.428571428571429
query_id: PLAIN-2208
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-1666
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2219


PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2699
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2709
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2719
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-2729
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2499
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2739
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-2579
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2749
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-258
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-2759
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2509
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2769
PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-2589
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2779
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2789
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-279
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2799
PPPPPPPPPPPPPPPPPP: 11.428571428571429
query_id: PLAIN-2599
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2809
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2519
PPPPPPPPPPPPP

PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-3140
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-3150
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2819
PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-3160
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2829
PPPPPPPPPPPPPPPPPP: 8.0
query_id: PLAIN-3170
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-3180
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2839
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2849
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-319
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-3190
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2859
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-2869
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-32
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-3200
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-3210
PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-3220
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-3230
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-3240
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-3250
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-3260
PPPPPPPP

query_id: PLAIN-1352
PPPPPPPPPPPPPPPPPP: 11.428571428571429
query_id: PLAIN-1362
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-319
PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-1373
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-3190
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-1386
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1397
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-1408
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1418
PPPPPPPPPPPPPPPPPP: 11.428571428571429
query_id: PLAIN-142
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-32
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-1428
PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-1440
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3200
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-1451
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-3210
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-1462
PPPPPPPPPPPPPPPPPP: 8.0
query_id: PLAIN-1472
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-3220
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-1484
PPPPPPPPPPPPPPPPPP: 26.666666666666668

PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-206
PPPPPPPPPPPPPPPPPP: 11.428571428571429
query_id: PLAIN-2060
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-3451
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2070
PPPPPPPPPPPPPPPPPP: 20.0
query_id: PLAIN-2080
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-2091
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-2101
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2112
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-3461
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-2123
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-2133
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-2144
PPPPPPPPPPPPPPPPPP: 8.0
query_id: PLAIN-2155
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-216
PPPPPPPPPPPPPPPPPP: 6.666666666666667
query_id: PLAIN-2166
PPPPPPPPPPPPPPPPPP: 13.333333333333334
query_id: PLAIN-2176
PPPPPPPPPPPPPPPPPP: 4.0
query_id: PLAIN-2186
PPPPPPPPPPPPPPPPPP: 10.0
query_id: PLAIN-2196
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-22
PPPPPPPPPPPPPPPPPP: 2

query_id: PLAIN-3013
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3025
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3036
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-304
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3052
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3062
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-3073
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3084
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3096
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3114
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3130
PPPPPPPPPPPPPPPPPP: 16.0
query_id: PLAIN-3140
PPPPPPPPPPPPPPPPPP: 40.0
query_id: PLAIN-3150
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-3160
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-3170
PPPPPPPPPPPPPPPPPP: 0
query_id: PLAIN-3180
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-319
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-3190
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-32
PPPPPPPPPPPPPPPPPP: 80.0
query_id: PLAIN-3200
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PLAIN-3210
PPPPPPPPPPPPPPPPPP: 26.666666666666668
query_id: PL

In [26]:
final.groupby('model').describe()

avgPrec                                               \
                         count      mean       std  min       25%       50%   
model                                                                         
CosineSparse             324.0  0.203769  0.220625  0.0  0.038656  0.118272   
Preclustering            324.0  0.156592  0.253440  0.0  0.000000  0.027619   
RandomProjectionHash     324.0  0.143102  0.206011  0.0  0.000000  0.051533   
RandomProjectionNoHash   324.0  0.159413  0.223377  0.0  0.000000  0.065271   

                                        NDCG            ...                 \
                             75%  max  count      mean  ...       75%  max   
model                                                   ...                  
CosineSparse            0.293919  1.0  324.0  0.178377  ...  0.246121  1.0   
Preclustering           0.215570  1.0  324.0  0.082650  ...  0.104996  1.0   
RandomProjectionHash    0.198103  1.0  324.0  0.099343  ...  0.128111  1.0   
RandomProjectionNoHash  0.214857  1.0  324.0  0.117112  ...  0.139152  1.0   

                         time                                          \
                        count      mean       std       min       25%   
model                                                                   
CosineSparse            324.0  0.869561  1.982708  0.077576  0.159341   
Preclustering           324.0  0.680099  1.887364  0.014184  0.027104   
RandomProjectionHash    324.0  0.906869  1.417020  0.319074  0.402399   
RandomProjectionNoHash  324.0  1.057852  1.688366  0.285207  0.370009   

                                                       
                             50%       75%        max  
model                                                  
CosineSparse            0.225195  0.875525  28.284481  
Preclustering           0.076001  0.641443  27.306431  
RandomProjectionHash    0.548329  0.863515  20.065686  
RandomProjectionNoHash  0.471256  1.015507  22.352090  

[4 rows x 24 columns]

In [ ]:
# qrel_file_path=r'../data/dev_gs.p'
# if __name__=='__main__':
#     e = Evaluation(qrel_file_path)
#     inverted_index_path = '../data/inverted_index.p'
#     global_tier_path = '../data/global_tier.p'
#     number_tier_path = '../data/number_tier.p'
#     stop_word_file_path = '../data/nfcorpus/raw/stopwords.large'
#     rare_words_file_path = '../data/rare_tokens.txt'
#     s = SearchModel(inverted_index_path, global_tier_path,number_tier_path, stop_word_file_path,rare_words_file_path)
#     query_df = pd.read_csv('../data/dev.all.queries.df')
#     gs_path = '../data/dev_gs.p'
#     res = e.Evaluation(query_df,s.init__search_model('TieredIndex'),500)

In [ ]:
# construct gs
# gs = defaultdict(list)
# with open('../data/dev.2-1-0.qrel','r') as f:
#     for line in f:
#         query_id,_, relavence,_=line.strip().split('\t')
#         gs[query_id].append(relavence)
# pickle.dump(dict(gs),open('../data/dev_gs.p','wb'))